## Problem 2

In this part we will again consider the network of the previous exercise, with weights according to transition matrix Lambda (see Lambda in the code below).
However, now we will simulate many particles moving around in the network in continuous time.

In [ ]:
##### Setup
import numpy as np
from numpy.random import choice
from copy import deepcopy
import matplotlib.pyplot as plt

# Define the mapping between 0,1,2,3,4 and o,a,b,c,d
mapping = {'o':0, 'a':1, 'b':2, 'c':3, 'd':4}
reverse_mapping = {0:'o', 1:'a', 2:'b', 3:'c', 4:'d'}

# Create Lambda
Lambda = [
    [0, 2/5, 1/5, 0, 0],
    [0, 0, 3/4, 1/4, 0],
    [1/2, 0, 0, 1/2, 0],
    [0, 0, 1/3, 0, 2/3],
    [0, 1/3, 0, 1/3, 0]]

# Define Q, the transition matrix associated to the jump chain associated to the CTMC
w = np.sum(Lambda, axis=1)
w_star = np.max(w)
Q = Lambda/w_star 
Q = Q + np.diag(np.ones(len(w))-np.sum(Q,axis=1))
print("Q matrix:\n", Q)


### a) Particle perspective
**a.i) If 100 particles all start in node a, what is the average time for a particle to return to
node a?**

N.B.: I interpreted this request as: "what is the average minimum time for at least one particle to leave and return to node a? I.e., what is the average minimum return time to node a, among the 100 particles?

In [ ]:
def return_time_100_particles(xi):
    '''
    This function simulates 100 particles doing a random walk, starting in node xi of a graph with associated transition matrix Q
    The algorithm stops when at least one particle has left and then has returned to xi.
    
    Returns:
        time: time instant at which one of the particles has returned to xi, after having left it
    '''
    n_nodes = len(Q)
    time = 0    # global clock time
    rate = 100 * w_star
    current_state = [ xi for _ in range(100) ]    # a list with 100 values; the i-th value is the node in which the i-th particle is, before the next jump
    t_next = -np.log(np.random.rand())/rate    # the random time to wait for the next transition is drawn from a rate-(100*w_star) exponential distribution
    
    while True:
        # Choose at random (uniformly) one of the 100 particles
        moving_particle = choice(range(100), size=1)[0]
        # Random jump:
        # the next state that moving_particle will visit will be extracted according to the probabilities
        # stored in the row of Q corresponding to moving_particle's current state, current_state[moving_particle]
        mov_part_curr_state = current_state[moving_particle]    # the last node that the moving particle was in
        new_state = choice(range(n_nodes), size=1, p=Q[mov_part_curr_state])[0]
        if new_state != mov_part_curr_state:    # jump
            current_state[moving_particle] = new_state
            time = time + t_next    # store the time instant of the current transition
            t_next = -np.log(np.random.rand())/rate
        else:    # do not jump, stay in the node: in this case, we still have to wait to jump
            t_next = t_next + -np.log(np.random.rand())/rate
        if new_state == xi and mov_part_curr_state != xi:    # stopping criterion
            return time    # a particle (in particular moving_particle) has returned to xi: the algorithm stops


In [ ]:
# Compute the average minimum return time to node a, among the 100 particles
# (the simulation is performed 1000 times, so the average is computed over 1000 samples)
n_rnd_walks = 1000
return_times_sample = np.array([return_time_100_particles(mapping['a']) for _ in range(n_rnd_walks)])
avg_return_time = return_times_sample.mean()

print("Average time for the first particle to return to a:", avg_return_time)

### b) Node perspective
**b.i) If 100 particles start in node o, and the system is simulated for 60 time units, what is
the average number of particles in the different nodes at the end of the simulation?**

In [ ]:
def simulate_100_particles_60_time_units(xi):
    '''
    This function simulates 100 particles doing a random walk for 60 time units, starting in node xi of a graph with associated transition matrix Q
    
    Returns:
        node_distribution: a list of quintuples;
                           node_distribution[i] is the distribution of the 100 particles on the len(Q)=5 nodes,
                           in the time interval ( transition_times[i], transition_times[i+1] )
        transition_times: a list of values;
                          transition_times[i], for i!=-1, is the time instant in which the i-th transition happens
                          transition_times[-1] is t=60.
    '''
    n_nodes = len(Q)
    time = 0    # global clock time
    time_limit = 60    # time units for which the simulation lasts
    transition_times = [0]    # this will store the time instants in which jumps are taken (i.e. in which the global clock ticks)
    rate = 100 * w_star
    t_next = -np.log(np.random.rand())/rate    # the random time to wait for the next transition is drawn from a rate-(100*w_star) exponential distribution

    node_distribution = [ [100 if i==xi else 0 for i in range(n_nodes)] ]    # a list which will contain lists, each containing five values (the n. of particles in each of the five nodes)
                                                                             # one list is appended when a particle is moved, to record the new distribution of particles
    while time + t_next <= time_limit:
        # Select a node at random, proportionally to the number of particles in the different nodes 
        current_node_distribution = node_distribution[-1]
        probs = [current_node_distribution[i] / 100 for i in range(n_nodes)]
        node = choice(range(n_nodes), size=1, p=probs)[0]
        # Random jump:
        # move a particle from node node to a neighboring node, with probabilities given by the row of Q corresponding to node node
        new_node = choice(range(n_nodes), size=1, p=Q[node])[0]
        new_node_distribution = deepcopy(current_node_distribution)
        new_node_distribution[node] -= 1    # NB: it doesn't matter if node and new_node coincide
        new_node_distribution[new_node] += 1
        node_distribution.append(new_node_distribution)

        time = time + t_next    # time instant of the jump just occurred
        transition_times.append(time)
        
        t_next = -np.log(np.random.rand())/rate    # the random time to wait for the next transition
        
    else:
        # t=60 seconds have passed; the simulation stops
        # Append the final distribution to node_distribution and 60 to transition_times
        # (this is done to correctly plot the n. of particles in each node during the simulation)
        node_distribution.append(node_distribution[-1])
        transition_times.append(60)
        
    return (node_distribution, transition_times)


In [ ]:
# Compute the average final (t=60) distribution of the 100 particles over the 5 nodes of the network
# (the simulation is performed 200 times, so the average is computed over 200 samples)
n_rnd_walks = 200

distribution_transitions_sample = [simulate_100_particles_60_time_units(mapping['o']) for _ in range(n_rnd_walks)]    # 200 couples
distribution_sample = [i[0] for i in distribution_transitions_sample]    # a list containing 200 lists; the i-th inner list contains node_distribution of the i-th simulation
transition_times_sample = [i[1] for i in distribution_transitions_sample]    # a list containing 200 lists; the i-th inner list contains transition_times of the i-th simulation

final_distribution_sample = []    # this list will contain only the distributions at t=60 of the 200 simulations
for d in distribution_sample:
    final_d = d[-1]
    final_distribution_sample.append(final_d)
avg_final_distribution = np.array(final_distribution_sample).mean(axis=0)

print("Average final distribution of particles over the nodes", avg_final_distribution)

Average final distribution of particles over the nodes [18.558 14.695 22.425 22.167 22.155] $\approx 100 \overline{\pi}$

**b.ii) Illustrate the simulation above with a plot showing the number of particles in each node during the simulation time.**

N.B.: I focus on just one of the 200 simulations performed previously, for instance the first one

In [ ]:
# Take node_distribution and transition_times of the first of the 200 simulations
node_distribution = np.array(distribution_sample[0])
transition_times = deepcopy(transition_times_sample[0])
    
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
for node in range(len(Q)):
    ax.plot(transition_times, node_distribution[:,node], label=f'node {reverse_mapping[node]}')    # label=f'node {reverse_mapping[node]}'
    
ax.legend(prop={'size': 20})

**b.iii) Compare the simulation result in the first point above with the stationary distribution of the continuous-time random walk followed by the single particles.**

In [ ]:
# The stationary distribution pi bar has been already computed in problem 1;
# here I just repeat that code

# Find pi bar, the (unique) stationary probability vector
values,vectors = np.linalg.eig(Q.T)
index = np.argmax(values.real)
pi_bar = vectors[:,index].real
pi_bar = pi_bar/np.sum(pi_bar)
print("pi_bar =", pi_bar)

As noticed previously, it seems that the average final distribution is approximately $100 \overline{\pi}$. This result has been justified in the report.